In [12]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pymupdf
import pandas as pd

In [2]:
base_model_id = "speakleash/Bielik-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    # llm_int8_enable_fp32_cpu_offload=True,
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
evaluated_ft_model = PeftModel.from_pretrained(base_model, "bielik-legal-finetune/checkpoint-400")

In [4]:
# load eval data
path = 'evaluation_data/umowa wzór.pdf'

all_data = pymupdf.open(path)
all_pages = []
for document in all_data:
    text_page = document.get_textpage().extractText()
    #chunks_per_page = splitter.split_text(text_page)
    all_pages.append(text_page)

print(len(all_pages))
base_context = all_pages[0]

3


In [5]:
def build_prompt(context: str, question: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "Jesteś prawniczym asystentem, ktory precyzyjnie odpowiada na pytania użytkowników",
        },
        {
            "role": "user",
            "content": f"Mając poniższy kontekst {context} odpowiedz proszę na pytanie: {question}"
        }

    ]
    return messages

In [6]:
# query = input("Enter your prompt: ")
#       messages = [
#          {"role": "user", "content": str(query)}
#       ]
#       encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
#       generated_ids = model.generate(encodeds, max_new_tokens=300, do_sample=False)
#       decoded = tokenizer.batch_decode(generated_ids)

In [38]:
eval_questions = [
    "Jakie są strony postępowania?",
    "Jakie są najważniejsze postanowienia niniejszej umowy?",
    "Jakie są najważniejsze daty w poniższej umowie?",
    "Jaki jest adres firmy?",
    "Czy umowa zawiera klauzule abuzywne?",
    "Jaka jest kwota zadatku?",
    "W jaki sposób przedsiębiorca może udowodnić, że nie znał danego publicznego zapewnienia i rozsądnie nie mógł o nim wiedzieć?"
]

In [39]:
eval_questions_df = pd.DataFrame(eval_questions, columns=['question'])
eval_questions_df.to_csv('eval_questions.csv', sep=';', index=False)

In [46]:
def inference(question: str):
    model_input = eval_tokenizer.apply_chat_template(build_prompt(context=base_context, question=question), return_tensors="pt").to("cuda")
    with torch.no_grad():
        generated_ids = evaluated_ft_model.generate(model_input, max_new_tokens=600, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
        #generated_ids_base = base_model.generate(model_input, max_new_tokens=500, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
        decoded = eval_tokenizer.batch_decode(generated_ids)
        #res = eval_tokenizer.decode(evaluated_ft_model.generate(model_input, max_new_tokens=600, do_sample=False, repetition_penalty=1.15)[0], skip_special_tokens=True)
    
    #decoded_base = eval_tokenizer.batch_decode(generated_ids_base)
    
    # Evaluated model
    looked_word = '[/INST]'
    inst_end = decoded[0].index(looked_word)
    answer = decoded[0][inst_end+len(looked_word):]
    print('*' * 20)
    print(answer)
    return answer


# print('#' * 20)
# print(res)
# Base model
# inst_end = decoded_base[0].index(looked_word)
# answer_base = decoded_base[0][inst_end+len(looked_word):]
# print('*' * 20)
# print(answer_base)

In [47]:
eval_questions_df['bielik-v1-ft'] = eval_questions_df['question'].apply(inference)

********************
 Stronami postępowania są AKANT Sp. z o.o. jako wykonawca oraz zamawiający, którym jest osoba fizyczna lub prawna, która zleca wykonanie usługi. </s>
********************
 Najważniejsze postanowienia niniejszej umowy to:

1. Przedmiotem umowy jest modernizacja mieszkania/domu poprzez montaż systemów osłonowych okien.
2. Umowa obejmuje poniższe systemy osłonowe: 1. (Produkt, ilość).
3. Wykonawca zapewnia niezbędne materiały, wyroby bądź produkty do wykonania przedmiotu umowy.
4. Przedmiot umowy będzie wykonany na indywidualne zamówienie Zamawiającego, pod wymiar, przy uwzględnieniu 
zindywidualizowanych potrzeb Zamawiającego i na podstawie zaakceptowanej oferty przez Zamawiającego.
5. Zamawiający zobowiązuje się do zapewnienia Wykonawcy dostępu do obiektu i miejsca prowadzenia prac.
6. Wykonanie usługi nastąpi w terminie 14-28 dni roboczych od dnia wpłaty zadatku.
7. Należność za przedmiot umowy ustala się na kwotę (…..)zł/brutto, z czego zadatek wynosi (…..) zł/bru

In [48]:
eval_questions_df.head()

,question,bielik-v1-ft
0,Jakie są strony postępowania?,Stronami postępowania są AKANT Sp. z o.o. jak...
1,Jakie są najważniejsze postanowienia niniejsze...,Najważniejsze postanowienia niniejszej umowy ...
2,Jakie są najważniejsze daty w poniższej umowie?,Najważniejsze daty w poniższej umowie to: \n1...
3,Jaki jest adres firmy?,Adres firmy: AKANT Sp. z o.o. 75-213 Koszali...
4,Czy umowa zawiera klauzule abuzywne?,"Tak, umowa zawiera klauzule abuzywne. Przykła..."


In [49]:
eval_questions_df.to_csv('eval_questions.csv', sep=';', index=False)

In [ ]:
# how messages look for model
# print(eval_tokenizer.decode(model_input[0]))

In [ ]:
# evaluated_ft_model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(
#         evaluated_ft_model.generate(**model_input, max_new_tokens=600, repetition_penalty=1.15)[0],
#         skip_special_tokens=True))